In [21]:
from pprint import pprint
from collections import namedtuple

In [2]:
train_file="./toy_train.txt"

In [23]:
word_info=namedtuple("word_info","id word pos head rel".split())

In [24]:
data=[]
sentence=[]

for line in open(train_file,"r",encoding="utf-8"):
    line=line[:-1]
    
    if len(line)!=0:
        ID,word,base, pos, pos1,_,head,rel=line.split("\t")
        ID=int(ID)-1
        head=int(head)-1
        
        sentence.append(word_info(ID,word,pos,head,rel))
    else:
        data.append(sentence)
        sentence=[]
        

In [26]:
pprint(data)

[[word_info(id=0, word='rolls-royce', pos='NNP', head=3, rel='DEP'),
  word_info(id=1, word='motor', pos='NNP', head=3, rel='DEP'),
  word_info(id=2, word='cars', pos='NNPS', head=3, rel='DEP'),
  word_info(id=3, word='inc.', pos='NNP', head=4, rel='NP-SBJ'),
  word_info(id=4, word='said', pos='VBD', head=-1, rel='ROOT'),
  word_info(id=5, word='it', pos='PRP', head=6, rel='NP-SBJ'),
  word_info(id=6, word='expects', pos='VBZ', head=4, rel='SBAR'),
  word_info(id=7, word='its', pos='PRP$', head=9, rel='DEP'),
  word_info(id=8, word='u.s.', pos='NNP', head=9, rel='DEP'),
  word_info(id=9, word='sales', pos='NNS', head=11, rel='NP-SBJ'),
  word_info(id=10, word='to', pos='TO', head=11, rel='DEP'),
  word_info(id=11, word='remain', pos='VB', head=6, rel='S'),
  word_info(id=12, word='steady', pos='JJ', head=11, rel='ADJP'),
  word_info(id=13, word='at', pos='IN', head=11, rel='PP'),
  word_info(id=14, word='about', pos='IN', head=16, rel='QP'),
  word_info(id=15, word='1,200', pos='CD', h

In [46]:
actions=[]

for sent in data:
    stack=[word_info(-1,"ROOT","ROOT",None,None)]
    buffer=sent[:]
    while len(buffer)>0 and len(stack)>0:
       
        if len(stack)>2:
            next_child_id=next_child(stack[-1],buffer)
        
            while next_child_id!=0:
                stack,buffer,actions=swap_next_child(stack,buffer,actions,next_child_id)
                next_child_id=next_child(stack[-1],buffer)
                
            if stack[-1].head==stack[-2].id:
                _=stack.pop(-1)
                actions.append("right-reduce")
                print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            elif stack[-2].head==stack[-1].id:
                _=stack.pop(-2)
                actions.append("left-reduce")
                print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            else:
                stack.append(buffer.pop(0))
                actions.append("shift")
                print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
        elif len(stack)>1:
            next_child_id=next_child(stack[-1],buffer)
        
            while next_child_id!=0:
                stack,buffer,actions=swap_next_child(stack,buffer,actions,next_child_id)
                next_child_id=next_child(stack[-1],buffer)
                
            stack.append(buffer.pop(0))
            actions.append("shift")
            print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
        else:
            stack.append(buffer.pop(0))
            actions.append("shift")
            print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
    while len(buffer)==0 and len(stack)>2 and len(stack)>2:
        if stack[-1].head==stack[-2].id:
            _=stack.pop(-1)
            actions.append("right-reduce")
            print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                        " ".join([wi.word for wi in stack]),
                                        " ".join([wi.word for wi in buffer])))
        elif stack[-2].head==stack[-1].id:
            _=stack.pop(-2)
            actions.append("left-reduce")
            print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                        " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))

shift           [    ROOT rolls-royce] [motor cars inc. said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
shift           [ROOT rolls-royce motor] [cars inc. said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
shift           [ROOT rolls-royce motor cars] [inc. said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
shift           [ROOT rolls-royce motor cars inc.] [said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
left-reduce     [ROOT rolls-royce motor inc.] [said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
left-reduce     [ROOT rolls-royce inc.] [said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
left-reduce     [           ROOT inc.] [said it expects its u.s. sales to remain steady at about 1,200 cars in 1990 .]
shift           [      ROOT inc. said] [it expects its u.s. sales to remain steady at about 1,200 cars in 1

In [36]:
def swap_next_child(stack,buffer,actions,next_child_id):
    while stack[-1].id!=next_child_id:
        stack.append(buffer.pop(0))
        actions.append("shift")
        print("{:15} [{:>20}] [{:40}]".format("shift",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    while stack[-2].id!=stack[-1].head:
        buffer.insert(0,stack.pop(-2))
        actions.append("swap")
        print("{:15} [{:>20}] [{:40}]".format("swap",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    return stack,buffer,actions

In [30]:
def next_child(word,buffer):
    for wi in buffer:
        if wi.head==word.id:
            return wi.id
    return 0

In [47]:
actions

['shift',
 'shift',
 'shift',
 'shift',
 'left-reduce',
 'left-reduce',
 'left-reduce',
 'shift',
 'shift',
 'shift',
 'swap',
 'shift',
 'right-reduce',
 'shift',
 'shift',
 'shift',
 'shift',
 'shift',
 'swap',
 'swap',
 'swap',
 'swap',
 'shift',
 'shift',
 'shift',
 'swap',
 'swap',
 'shift',
 'right-reduce',
 'shift',
 'right-reduce',
 'right-reduce',
 'shift',
 'right-reduce',
 'shift',
 'right-reduce',
 'shift',
 'shift',
 'shift',
 'shift',
 'swap',
 'swap',
 'shift',
 'shift',
 'right-reduce',
 'right-reduce',
 'right-reduce',
 'right-reduce',
 'shift',
 'shift',
 'right-reduce',
 'right-reduce',
 'right-reduce',
 'right-reduce',
 'shift',
 'right-reduce',
 'left-reduce']

In [40]:
actions=[]
actions.append("shift")